# 실습 06: 이미지 전처리로 OCR 정확도 향상

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab06_이미지전처리.ipynb)

## 🎯 학습 목표
- 이미지 품질 평가
- 노이즈 제거, 이진화, 기울기 보정
- 전처리 전후 OCR 성능 비교

## ⏱️ 소요 시간: 40분
## 📊 난이도: ⭐⭐⭐⭐☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# 환경 설정
!pip install -q git+https://github.com/leecks1119/document_ai_lecture.git
!apt-get install -y tesseract-ocr tesseract-ocr-kor


In [ ]:
# 노이즈가 있는 테스트 이미지 생성
from docai_course.preprocessing import DocumentPreprocessor
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np

# 한글 폰트 로드
try:
    font = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 20)
except:
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 20)
    except:
        font = ImageFont.load_default()

# 깨끗한 이미지 생성
img = Image.new('RGB', (600, 300), color='white')
draw = ImageDraw.Draw(img)
text = "계약서\n\n계약일자: 2025년 1월 15일\n계약금액: 1,500,000원\n담당자: 홍길동 (02-1234-5678)"
draw.text((30, 30), text, fill='black', font=font)

# OpenCV 변환 및 노이즈 추가
img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
noise = np.random.normal(0, 25, img_cv.shape).astype(np.uint8)
noisy_img = cv2.add(img_cv, noise)

# 기울기 추가 (5도)
rows, cols = noisy_img.shape[:2]
M = cv2.getRotationMatrix2D((cols/2, rows/2), 5, 1)
rotated_img = cv2.warpAffine(noisy_img, M, (cols, rows), borderValue=(255,255,255))
cv2.imwrite('noisy_document.jpg', rotated_img)

print("✅ 노이즈 이미지 생성 완료!")


In [ ]:
# 적응적 전처리 실행
preprocessor = DocumentPreprocessor(debug_mode=True)

processed_img, binary_img, quality_metrics = preprocessor.adaptive_preprocessing_pipeline(
    image_path='noisy_document.jpg'
)

print("\n📊 이미지 품질 지표:")
for key, value in quality_metrics.items():
    print(f"  {key}: {value}")


In [ ]:
# 전처리 전후 OCR 비교
import pytesseract
import Levenshtein

ground_truth = "계약서 계약일자: 2025년 1월 15일 계약금액: 1,500,000원 담당자: 홍길동 (02-1234-5678)"

# OCR 실행
original = cv2.imread('noisy_document.jpg')
original_text = pytesseract.image_to_string(original, lang='kor+eng')
processed_text = pytesseract.image_to_string(binary_img, lang='kor+eng')

# 정확도 계산
def calc_accuracy(pred, truth):
    distance = Levenshtein.distance(pred, truth)
    return max(0, (1 - distance / max(len(pred), len(truth))) * 100)

original_acc = calc_accuracy(original_text, ground_truth)
processed_acc = calc_accuracy(processed_text, ground_truth)

print(f"\n📊 정확도 개선:")
print(f"  원본: {original_acc:.2f}%")
print(f"  전처리 후: {processed_acc:.2f}%")
print(f"  개선: +{processed_acc - original_acc:.2f}%p")
print(f"\n✅ 전처리로 OCR 정확도 향상!")
